## Hashing

<font size = "3">

- When lists were ordered, we could take advantage of the structure to create a search algorithm with $\mathcal{O}(\log (n))$ complexity

- Using **hashing** we can define a search algorithm that is (most of the time) $\mathcal{O}(1)$

- This technique is used by Python dictionaries, for example. (Recall the last test from "lecture04_python_data_structures.ipynb")

- A **hash table** stores items using (key, value) pairs. Depending on the key, the value is stored in a **slot** (position) of the hash table.

In [ ]:
dsci_531 = {"Kwatcho" : "Data Sci",
    "Kejin" : "Data Sci", 
    "Zach" : "Poli Sci"}

print(dsci_531.keys())
print(dsci_531.values())

In [ ]:
print(dsci_531["Kwatcho"])

In [ ]:
## What slot is everyone in?
## Use the built-in "hash" funciton

kwatcho_slot = hash("Kwatcho")
kejin_slot = hash("Kejin")
zach_slot = hash("Zach")
print(kwatcho_slot)
print(kejin_slot)
print(zach_slot)

<font size = "3">

(Note: this is not the whole picture. A dictionary with 3 items does not create a data structure with 7 quintillion slots. So a further transformation is done with these huge numbers)

<font size = "3">

- The **hash function** defines the mapping between a key and which slot the item is stored in.

- We'll start with an example of an empty hash table with 11 slots.

<div style="text-align: center;">
  <img src="files/hashtable.png" alt="Centered image" width = "475">
  <figcaption><font size = "1"> Miller, Randum, Yasinovskyy (Problem Solving with Algorithms and Data Structures using Python)</figcaption>
</div>

<br>

<font size = "3">

- We'll keep it simple, and make the keys identical to the items.

- For the hash function, we'll take the remainder after dividing by 11 (the number of slots)

- We'll add the keys (same as the items) 54, 26, 93, 17, 77, 31.

In [ ]:
for key in [54, 26, 93, 17, 77, 31]:
    print(str(key) + ": " + str(key % 11))

<font size = "3">

- According to the hash function, this is where the items should be located:

<div style="text-align: center;">
  <img src="files/hashtable2.png" alt="Centered image" width = "475">
  <figcaption><font size = "1"> Miller, Randum, Yasinovskyy (Problem Solving with Algorithms and Data Structures using Python)</figcaption>
</div>

<br>

<font size = "3">

- Now, 6 of 11 of the slots are occupied. Thus, the **load factor** $\lambda = \frac{6}{11}$

- If we want to search for a key, we compute the remainder after division by 11, and then look in that slot. That's $\mathcal{O}(1)$!

- However, what if we wanted to add an additional item with key 44? The hash function says it should be in slot 0, which is occupied. This is a **collision** (or clash)

- A **perfect hash function** maps each key into a unique slot. This is too ambitious, so instead we choose some method to perform **collision resolution**


### Examples of other hash functions

<font size = "3">

**Folding method**

- Divide keys into pieces of equal size (the last piece may not be of equal size). Then add them together to get hash value.

- Example: phone number "414-573-0094" has the hash value

$$ 41 + 45 + 73 + 00 + 94 = 253$$

- If we want to fit it into a hash table with 11 slots, we could then divide by 11 and take the remainder (slot 0 in this case.)

- Could also reverse every other piece before summing:

$$41 + 54 + 73 + 00 + 94 = 262$$

- Note that 00 piece was one of those that was "reversed". Under this method, the phone number would go in slot 9.

**Midsquare method**

- If the key is an integer, square it, and extract the middle $r$ digits. Can use the remainder divided by the number of slots.

- For example, if the key is 44. Then $44^2 = 1,936$. If we take the middle two digits, then the hash is 93. (Slot 5 for an 11-element hash table.)

<font size = "3">

**Hashing strings**

- We can use the `ord` function to map characters to integers.

- Suppose our key is "cat"

In [ ]:
print(ord("c"))
print(ord("a"))
print(ord("t"))

<font size = "3">

- We can add all these together (and compute remainder after dividing by hash table size )

$$99 + 97 + 116 = 312$$

- For 11-element hash-table, this goes in slot 4

- Here's an implementation of the hash function

In [ ]:
def hash_str(a_string, table_size):
    return sum([ord(c) for c in a_string]) % table_size

print(hash_str("cat", 11))
print(hash_str("lion", 11))
print(hash_str("blue-footed booby", 11))

<font size = "3">

- Anagrams will always lead to a hash collision:

In [ ]:
print(hash_str("tea", 11))
print(hash_str("eat", 11))

<font size = "3">

- We can compute a weighted sum, where the weights correspond to position in the string

In [ ]:
print((1*ord("t") + 2*ord("e") + 3*ord("a")) % 11)
print((1*ord("e") + 2*ord("a") + 3*ord("t")) % 11)

<font size = "3">

- Implementation of weighted hashing of a string:

In [ ]:
def hash_str_weighted(a_string, table_size):
    n = len(a_string)
    return sum([ord(c)*(i+1) for c, i in zip(a_string, range(n))]) % table_size

In [ ]:
print(hash_str_weighted("cat", 11))
print(hash_str_weighted("lion", 11))
print(hash_str_weighted("blue-footed booby", 11)) # collision with "cat"
print(hash_str_weighted("tea", 11))
print(hash_str_weighted("eat", 11))


### Collision Resolution

<font size = "3">

- When two keys are mapped to the same slot by the hash function, we have a collision and need to resolve it.

- One resolution approach is **open addressing** - if the intended slot is occupied, find some other empty slot in the hash table.


- This is also known as **rehashing**

- With **linear probing** we can systematically visit each slot one at a time.

- Here's an example with a hash table of size 11:

In [ ]:
hash_table = [None] * 11
item_stream = [54, 26, 93, 17, 77, 31, 44, 55, 20]

for item in item_stream:
    slot = item % 11
    print(str(item) + "'s slot is: " + str(slot))
    if hash_table[slot] is None:
        hash_table[slot] = item 
    else:
        for k in range(len(hash_table) - 1):
            slot = (slot + 1) % 11
            if hash_table[slot] is None:
                hash_table[slot] = item 
                break
print("*******")
print(hash_table)

<font size = "3">

- When using linear probing, there is a tendency for **clustering** -- items with the same hash value are clustered in the same region of the hash table.

- In the example above, we see clustering for the keys 77, 44, 55.

- When we have clustering, this can hurt the efficiency of resolving collisions for other items (e.g. consider the collision resolution when we added 20 to the hash table)

- To help mitigate clustering, we could linearly probe but search for an empty slot in jumps of 3.

In [ ]:
hash_table = [None] * 11
item_stream = [54, 26, 93, 17, 77, 31, 44, 55, 20]

for item in item_stream:
    slot = item % 11
    print(str(item) + "'s slot is: " + str(slot))
    if hash_table[slot] is None:
        hash_table[slot] = item 
    else:
        for k in range(len(hash_table) - 1):
            slot = (slot + 3) % 11
            if hash_table[slot] is None:
                hash_table[slot] = item 
                break
print("*******")
print(hash_table)

<font size = "3">

- **Note:** This approach works because the table size (11) and the skip size (3) are *relatively prime*, i.e. $\textrm{gcd}(3, 11) = 1$. We could not use a skip size of 3 if the hash table had 12 slots.

- Another approach is **quadratic probing** where we try slot + $1^2$ = slot + 1, then slot + $2^2$ = slot + 4, then slot + $3^2$ = slot + 9, etc.

In [ ]:
hash_table = [None] * 11
item_stream = [54, 26, 93, 17, 77, 31, 44, 55, 20]

for item in item_stream:
    slot = item % 11
    print(str(item) + "'s slot is: " + str(slot))
    if hash_table[slot] is None:
        hash_table[slot] = item 
    else:
        for k in range(len(hash_table) - 1):
            re_slot = (slot + (k+1)**2) % 11
            if hash_table[re_slot] is None:
                hash_table[re_slot] = item 
                break
print("*******")
print(hash_table)

<font size = "4">

- **Question:** would this approach work with a hash table of size 12?

<font size = "3">

- An alternative to **open addressing** is chaining. Just allow multiple values in a single slot:

<div style="text-align: center;">
  <img src="files/chaining.png" alt="Centered image" width = "475">
  <figcaption><font size = "1"> Miller, Randum, Yasinovskyy (Problem Solving with Algorithms and Data Structures using Python)</figcaption>
</div>

<br>

<font size = "3">


### Implementation of a Hash Table

In [ ]:
class HashTable:
    def __init__(self, size):
        self.size = size 
        self.slots = [None] * self.size 
        self.data = [None] * self.size

    def put(self, key, data):
        hash_value = self.hash_function(key, len(self.slots))

        if self.slots[hash_value] is None:
            self.slots[hash_value] = key 
            self.data[hash_value] = data 
        else:
            if self.slots[hash_value] == key:
                self.data[hash_value] = data # replace
            else:
                next_slot = self.rehash(hash_value, len(self.slots))
                while (
                    self.slots[next_slot] is not None
                    and self.slots[next_slot] != key
                ):
                    next_slot = self.rehash(next_slot, len(self.slots))

                    ###### added this next check to avoid infinite loop
                    if next_slot == hash_value:
                        raise KeyError("Cannot hold more items!")

                if self.slots[next_slot] is None:
                    self.slots[next_slot] = key 
                    self.data[next_slot] = data 
                else:
                    self.data[next_slot] = data 

    def hash_function(self, key, size):
        return key % size 

    def rehash(self, old_hash, size):
        return (old_hash + 1) % size

    def get(self, key):
        start_slot = self.hash_function(key, len(self.slots))

        position = start_slot 
        while self.slots[position] is not None:
            if self.slots[position] == key:
                return self.data[position]
            else:
                position = self.rehash(position, len(self.slots))
                if position == start_slot:
                    return None 
    
    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, data):
        self.put(key, data)

In [ ]:
h = HashTable(11)
h[54] = "cat"
h[26] = "dog"
h[93] = "lion"
h[17] = "tiger"
h[77] = "bird"
h[31] = "cow"
h[44] = "goat"
h[55] = "pig"
h[20] = "chicken"

print(h.slots)
print(h.data)

In [ ]:
print(h[20])
print(h[17])
h[20] = "duck"
print(h[20])
print(h.data)
print(h[99])

<font size = "3">

- This is a dumb example...why does the key 54 mean "cat"?

- Let's adjust this to make it more in line with a Python dictionary:

#### Cost of Hashing

<font size = "3">

- Optimistically, the cost of searching a hash table is $O(1)$. 

- If there are **no** collisions, then it *is* $O(1)$.

- As more collisions occur, we will have to expand our search to other slots. The number of collisions is related to the **load factor** $\lambda$, the ratio of occupied slots to total slots.

- Larger $\lambda$ means higher probability we will have a collision and will need to search more places.

**Case 1: Linear probing**

- Item belongs to hash table: average cost $\approx \frac{1}{2}\left(1 + \frac{1}{1-\lambda}\right)$

- Item is not in hash table: average cost $\approx \frac{1}{2}\left(1 + \frac{1}{(1-\lambda)^2}\right)$

**Case 2: Chaining**

- Item belongs to hash table: average cost $\approx 1 + \frac{\lambda}{2}$

- Item is not in hash table: average cost $\approx \lambda$